In [1]:
using Pkg; Pkg.activate(".")
using BarnesDoubleGamma, BenchmarkTools

  Activating project at `~/Documents/Recherche/projet_these/code/BarnesDoubleGamma`


In [ ]:
import BarnesDoubleGamma: LogBDoubleGamma

setprecision(BigFloat, 256)

β = big"0.8" + big"0.5"*im
z = big"0.9" + big"0.2"*im

lBDG = LogBDoubleGamma(β)

# @btime BarnesDoubleGamma.rest_RMN($z, $lBDG.cache)
# @btime BarnesDoubleGamma.log_Barnes_GN($z, $lBDG.cache)

function loggamma_stirling(x)
    log(2π)/2 + (x - 1/2)*log(x) - x + 1/(12x) - 1/(360x^3)  # Add terms as needed
end

# @btime loggamma_stirling(z + 100 * lBDG.cache.τ)
# @btime loggamma(z + 100 * lBDG.cache.τ)


function compute_lgammas!(lgams::Vector{T}, z::T, τ::T) where {T}
    N = length(lgams)
    nthreads = Threads.nthreads()
    chunk_size = cld(N, nthreads)

    Threads.@threads for thread_id in 1:nthreads
        start = (thread_id - 1) * chunk_size + 1
        stop  = min(thread_id * chunk_size, N)
        @inbounds for m in start:stop
            lgams[m] = loggamma(z + m * τ)
        end
    end
end

@btime Threads.@threads for m in 1:2100
    lgams[m] = loggamma(z + m * lBDG.cache.τ)
end

@btime for m in 1:2100
    lgams[m] = loggamma(z + m * lBDG.cache.τ)
end


  24.608 ms (105728 allocations: 4.50 MiB)
  25.625 ms (121503 allocations: 5.45 MiB)


In [31]:
2100 * 23.876e-6

0.0501396

1 computation without caching powers of $1/(N\tau)$: 591 microsec.

In [41]:
setprecision(BigFloat, 30, base=10)
τ = sqrt(3)
lbdg = LogBDoubleGamma(τ);

In [42]:
cache = lbdg.cache
M, N = cache.M, cache.N
z = τ
BarnesDoubleGamma.rest_RMN(z, cache)

0.00013872882747994979 - 0.0im

In [43]:
BarnesDoubleGamma.rest_RMN_old(M, N, z, τ)

0.00013872882747994979

In [44]:
GN = BarnesDoubleGamma.log_Barnes_GN(N, z, cache)

0.39733576087181266 + 0.0im

In [45]:
GN_old = BarnesDoubleGamma.log_Barnes_GN_old(N, z, complex(τ))

0.3973357608718123 + 0.0im

In [46]:
G_old = BarnesDoubleGamma.log_barnesdoublegamma_old(M, N, z, complex(τ))

0.39805661700482153 + 0.0im

In [47]:
G = lbdg(z)

0.39805661700482153 + 0.0im

In [48]:
log_exact_val(τ) = (τ - 1) / 2 * log(2oftype(τ, π)) - log(τ) / 2
exact_val(τ) = (2oftype(τ, π))^((τ - 1) / 2) / sqrt(τ)

exp(G) - exact_val(τ)

-9.181910121114356e-9 + 0.0im